In [49]:
import pandas as pd
import numpy as np
import re
import time
import dill
from glob import glob
from csv_pkl_sql import save_it

## Import infection data

In [50]:
data.head(1)

,report_date,location,location_type,data_field,data_field_code,value,unit
0,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_local_cases,AR0001,0,cases


In [51]:
data_file_locations = glob('../zika/*/*/data/*.csv')
data = pd.concat([pd.read_csv(x)
                            for x in data_file_locations], axis=0).reset_index(drop=True)

data.drop(['time_period','time_period_type'], axis=1, inplace=True)

data['report_date'] = data.report_date.str.replace('_','-')       
data['report_date'] = pd.to_datetime(data.report_date)

# save_it(data, '03_infection_data_initial_import')

In [52]:
data.query("value>0").shape, data.shape

((37513, 7), (107940, 7))

In [9]:
data_locations.data_field.unique()

array(['cumulative_confirmed_local_cases',
       'cumulative_probable_local_cases',
       'cumulative_confirmed_imported_cases',
       'cumulative_probable_imported_cases',
       'cumulative_cases_under_study', 'cumulative_cases_discarded',
       'microcephaly_under_investigation', 'microcephaly_confirmed',
       'microcephaly_not', 'municipality_microcephaly',
       'microcephaly_fatal_under_investigation',
       'microcephaly_fatal_confirmed', 'microcephaly_fatal_not',
       'municipality_microcephaly_suspected', 'zika_reported',
       'zika_confirmed_laboratory', 'zika_confirmed_clinic',
       'zika_suspected', 'zika_suspected_clinic',
       'total_zika_new_suspected', 'total_zika_new_confirmed_pcr',
       'zika_new_confirmed_pcr_f', 'zika_new_confirmed_pcr_m',
       'efe_reported', 'zika_suspected_cumulative',
       'zika_suspected_pregnant_cumulative',
       'zika_confirmed_pcr_cumulative', 'zika_suspected_pregnant',
       'gbs_reported', 'gbs_zika_confirmed', 'gb